# On The Free Energy Penalty of Cavity Formation In Salt Solutions: Rethinking the Terms “Kosmotropic” and “Chaotropic”.

Stefan Hervø-Hansen<sup>a,*</sup> and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Force field parameter generation
For this project we are gonna utilize the [OPC](https://doi.org/10.1021/jz501780a) water model, [Sengupta & Merz parameters](https://doi.org/10.1021/acs.jcim.0c01390) for monatomic anions and cations, and finally the solute using the [Sage force field](https://doi.org/10.5281/zenodo.5214478).

In this notebook we will generate the force field parameters for the solute molecules one choose to study. In order to do so one must include the solute molecule in the variable `solutes` with the `Category` being a subdirectory within the `PDB_files` directory. If one wishes to use a different force field among the OpenFF Force Fields, one may edit the `forcefield` variable. The list of force fields can be obtained [here](https://github.com/openforcefield/openff-forcefields).


## Import of Python Modules & Auxiliary Functions

In [1]:
from openmm.app import PDBFile
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import parmed as pmd
import os

homedir = !pwd
homedir = homedir[0]
print(homedir)

/data2/stefan/Cavity-Formation


#### Solute list and choice of force field

In [2]:
## SETTINGS

# Solute list
solutes = {
    'methanol':   {'SMILES': 'CO',      'Category': 'n-alcohols'},
    'ethanol' :   {'SMILES': 'CCO',     'Category': 'n-alcohols'},
    'n-propanol': {'SMILES': 'CCCO',    'Category': 'n-alcohols'},
    'n-butanol' : {'SMILES': 'CCCCO',   'Category': 'n-alcohols'},
    'n-hexanol':  {'SMILES': 'CCCCCCO', 'Category': 'n-alcohols'},
    
    'methane':    {'SMILES': 'C',       'Category': 'n-alkanes'},
    'ethane' :    {'SMILES': 'CC',      'Category': 'n-alkanes'},
    'propane':    {'SMILES': 'CCC',     'Category': 'n-alkanes'},
    'n-butane' :  {'SMILES': 'CCCC',    'Category': 'n-alkanes'},
    'n-pentane':  {'SMILES': 'CCCCC',   'Category': 'n-alkanes'},
    'n-hexane':   {'SMILES': 'CCCCCC',  'Category': 'n-alkanes'},
}

# Force field (https://github.com/openforcefield/openff-forcefields)
forcefield = ForceField("openff-2.0.0.offxml")

#### Generate parameters

In [3]:
%cd -q $homedir
for solute, solute_properties in solutes.items():
    if not os.path.isfile('Force_fields/{}.itp'.format(solute)): # check if a force field file exist for the solute
        molecule = Molecule.from_smiles(solute_properties['SMILES'])
        pdbfile = PDBFile('PDB_files/{}/{}.pdb'.format(solute_properties['Category'], solute))
        omm_topology = pdbfile.topology
        off_topology = Topology.from_openmm(omm_topology, unique_molecules=[molecule])
        omm_system = forcefield.create_openmm_system(off_topology)
        parmed_structure = pmd.openmm.load_topology(omm_topology, omm_system, pdbfile.positions)
        parmed_structure.save("Force_fields/temporary.top", overwrite=True)
        
        with open("Force_fields/temporary.top", "r+") as f:
            lines = f.readlines()
        f.close()
        os.remove('Force_fields/temporary.top')
        
        with open('Force_fields/{}.itp'.format(solute), "w") as f:
            for i,line in enumerate(lines):
                if '[ defaults ]' in line:
                    for j in range(3):
                        lines[i+j] = ''
            f.seek(0)
            for line in lines[:-7]:
                f.write(line)
        f.close()